# Desarrollo del tercer proyecto de bases de datos II: Big Data & Spark

## Requerimientos Previos

Para replicar los experimentos acá tratados se debe contar con los siguientes programas y requisitos:

    - Sistema Operativo Linux
    - Python3
    - Jupyter 
    - PySpark
    - Scale
    - Java (JDK)
    - Openpyxl

## Imports

Para el desarrollo de este proyecto se hará uso de las librerías pandas como auxliar y el framework Spark por lo tanto hacemos los imports y variables de configuracion globales

In [15]:
import pandas as pd
import pyspark
from pyspark.sql.types import *

In [16]:
from pyspark.sql import SparkSession, Row, dataframe

In [4]:
import findspark
SPARK_PATH = '/home/bryan/spark'
findspark.init(SPARK_PATH)

POSTGRESQL_URL = "jdbc:postgresql://localhost/"
POSTGRESQL_USER = "postgres"
POSTGRESQL_PASSWORD = "root"

## Conexión con Spark

Se construye el metodo que nos permite hacer conexión con Spark, no recibe pero retorna un objeto de tipo SparkSession el cual usaremos para trabajar con Spark

In [5]:
def sparkSession():
    """
    This function builds a Spark Session
    return the main entry of a Spark DataFrame
    """
    spark = SparkSession \
      .builder \
      .appName("Test") \
      .config("spark.driver.extraClassPath", "postgresql-42.1.4.jar") \
      .config("spark.executor.extraClassPath", "postgresql-42.1.4.jar") \
      .getOrCreate()
    return spark

In [6]:
spark = sparkSession()#Definimos el objeto Spark

In [7]:
print(spark) #Si imprime bien, la conexion es exitosa

## Obtencion de los datos y conversión a DataFrame de Spark

Por medio de pandas se contacta a los archivos con los datos, y despues creamos los datas frames con Spark

In [23]:
df = pd.read_excel('Estadisticas.xlsx', engine='openpyxl') #Se lee el xlsx con pandas
oijEstadistics = spark.createDataFrame(df) #Se convierte a dataFrame

#Se define une esquema para decirle a Spark como debe leer los datos del xlsx
mySchema = StructType([StructField("Provincia, canton y distrito", StringType(), True)\
                      ,StructField("Poblacion de 15 años y más", StringType(), True)\
                      ,StructField("Tasa neta de participacion", StringType(), True)\
                      ,StructField("Tasa de ocupacion", StringType(), True)\
                      ,StructField("Tasa de desempleo abierto", StringType(), True)\
                      ,StructField("Porcentaje de poblacion economicamente inactiva", DoubleType(), True)\
                      ,StructField("Relacion de dependencia economica", DoubleType(), True)\
                      ,StructField("Sector \nPrimario", StringType(), True)\
                      ,StructField("Sector \nSecundario", StringType(), True)\
                      ,StructField("Sector Terciario", StringType(), True)])

df = pd.read_excel('INECEstadisticas.xlsx', engine='openpyxl') #Se lee el xlsx con pandas
inecEstadistics = spark.createDataFrame(df,schema=mySchema)#Se convierte a dataFrame


Como prueba de la obtención de los datos mostramos el estados de los dataFrames

In [26]:
oijEstadistics.show()
inecEstadistics.show()

+------+-----------+----------+-------------------+-----------+--------------------+-------------+------+------------+----------+------------+----------------+
|Delito|  SubDelito|     Fecha|               Hora|    Victima|          SubVictima|         Edad|Genero|Nacionalidad| Provincia|      Canton|        Distrito|
+------+-----------+----------+-------------------+-----------+--------------------+-------------+------+------------+----------+------------+----------------+
|ASALTO|ARMA BLANCA|2021-01-03|06:00:00 - 08:59:59|EDIFICACION|PANADERIA [EDIFIC...|Mayor de edad| MUJER|  COSTA RICA|  ALAJUELA|       UPALA|         BIJAGUA|
|ASALTO|ARMA BLANCA|2021-01-02|06:00:00 - 08:59:59|   VIVIENDA|NO APLICA [VIVIENDA]|Mayor de edad| MUJER|  COSTA RICA|     LIMON|       LIMON|          MATAMA|
|ASALTO|ARMA BLANCA|2021-01-07|12:00:00 - 14:59:59|    PERSONA|    PEATON [PERSONA]|Mayor de edad| MUJER|  COSTA RICA|  SAN JOSE|    SAN JOSE|        HOSPITAL|
|ASALTO|ARMA BLANCA|2021-01-08|03:00:00 